In [9]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import glob
import exif
import statistics
import numpy as np
import os

## Get GPS

Convert from (degrees, minutes, seconds) to decimal:

Degrees + Minutes/60 + Seconds/3600

Using the python exif tool; [intruction here](https://exif.readthedocs.io/en/latest/usage.html)

In [2]:

def get_gps(image_file_path):

    image = exif.Image(file)
    
    if not image.has_exif:
        return (None, None)
    
    lat = image.gps_latitude[0] + image.gps_latitude[1]/60. + image.gps_latitude[2]/3600.
    lat = lat if image.gps_latitude_ref == 'N' else -lat
    
    lon = image.gps_longitude[0] + image.gps_longitude[1]/60. + image.gps_longitude[2]/3600.
    lon = lon if image.gps_longitude_ref == 'E' else -lon
    
    # print(image.gps_latitude, image.gps_longitude)

    return (lat,lon)


In [20]:
pathname = "img/01/*.jpg"
images_01 = glob.glob(pathname)
num_files = len(images_01)
print(num_files)

# onlyfiles = next(os.walk(images_01))[2] #directory is your directory path as string
# print (len(onlyfiles))


5


In [114]:
pathname = "img/01/*.jpg"
images_01 = glob.glob(pathname)
num_files = len(images_01)
print()

lats, lons = [],[]
gps = np.zeros(shape=(num_files, 2),dtype=np.float64)
print(gps.shape)


for i, image in enumerate(images_01):
    with open(image, 'rb') as file:
        lat, lon = get_gps(file)  
        gps[i] = [lon, lat]
        

avg_gps = gps.mean(axis=0)
sd_gps = np.max(gps.std(axis=0))


print ('Average gps :{:}'.format(avg_gps))
print ('Stdev {:}m'.format(sd_gps))



(5, 2)
Average gps :[-122.64041222   37.89626722]
Stdev 1.5555555557966726e-05m


# Plot on EE Maps

In [4]:
import ee
import geemap

In [5]:
#ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [115]:
gps.tolist()

[[-122.64041666666667, 37.896275],
 [-122.64041666666667, 37.896275],
 [-122.64039444444445, 37.89623611111111],
 [-122.64041666666667, 37.896275],
 [-122.64041666666667, 37.896275]]

In [118]:
sample_avg = ee.Geometry.Point([avg_gps[0], avg_gps[1]])
buffer = sample_avg.buffer(10).bounds()
buffer.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-122.64052533011711, 37.89617726814456],
   [-122.64029875524955, 37.89617726814456],
   [-122.64029875524955, 37.89635721088176],
   [-122.64052533011711, 37.89635721088176],
   [-122.64052533011711, 37.89617726814456]]]}

In [119]:
polygon = ee.Geometry.Polygon(gps.tolist())
polygon.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-122.64041666666667, 37.896275],
   [-122.64041666666667, 37.896275],
   [-122.64039444444445, 37.89623611111111],
   [-122.64041666666667, 37.896275],
   [-122.64041666666667, 37.896275]]]}

In [128]:
Map = geemap.Map()
url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Satellite', attribution='Google')
Map.add_marker((avg_gps[1],avg_gps[0]),tooltip='sample', name='avg center')
Map.addLayer(polygon,
             {'color':'black'},
             name='samples')
Map.addLayer(buffer,
             {'color':'yellow'},
             name='buffer')
Map.setCenter(avg_gps[0], avg_gps[1], zoom=17)

Map

Map(center=[37.89626722222222, -122.64041222222222], controls=(WidgetControl(options=['position', 'transparent…

In [98]:
print (pointBuffer.getInfo())

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-142.10406374571147, -57.359767685932624], [-142.10340075959914, -57.359767685932624], [-142.10340075959914, -57.35940780045871], [-142.10406374571147, -57.35940780045871], [-142.10406374571147, -57.359767685932624]]]}


In [101]:
import ee
import geemap

Map = geemap.Map()

# Define a Polygon object.
polygon = ee.Geometry.Polygon(
    [[[-122.092, 37.424],
      [-122.086, 37.418],
      [-122.079, 37.425],
      [-122.085, 37.423]]])

# Define other inputs.
inputGeom = ee.Geometry.BBox(-122.085, 37.415, -122.075, 37.425)

# Apply the intersection method to the Polygon object.
polygonIntersection = polygon.intersection(**{'right': inputGeom, 'maxError': 1})

# Print the result to the console.
print('polygon.intersection(...) =', polygonIntersection.getInfo())

# Display relevant geometries on the map.
Map.setCenter(-122.085, 37.422, 15)
Map.addLayer(polygon,
             {'color': 'black'},
             'Geometry [black]: polygon')
Map.addLayer(inputGeom,
             {'color': 'blue'},
             'Parameter [blue]: inputGeom')
Map.addLayer(polygonIntersection,
             {'color': 'red'},
             'Result [red]: polygon.intersection')
Map

polygon.intersection(...) = {'type': 'Polygon', 'coordinates': [[[-122.085, 37.419000105397885], [-122.07900000000001, 37.425], [-122.085, 37.423], [-122.085, 37.419000105397885]]]}


Map(center=[37.422, -122.085], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…